In [1]:
import pandas as pd
import numpy as np
import os
import codecs
import seaborn as sns

c:\Users\mahes\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv("spam (1).csv",encoding='ISO-8859-1')

In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df=df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [6]:
import nltk 
from nltk import word_tokenize
nltk.download('punkt')

def token(input):
    word=[]
    for w in word_tokenize:
        word.append(w)
    return word 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mahes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
df['label_x'] = df.v1.map({'ham':0,'spam':1})

In [8]:

df.head()

,v1,v2,label_x
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [9]:
df=df.drop(['v1'],axis=1)

In [10]:
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

def text(input):
    Stop=stopwords.words('english')+['u','ur','4','2','im','dont','ure']
    nopunc=[char for char in input if char not in string.punctuation]

    nopunc=''.join(nopunc)

    return ' '.join([word for word in nopunc.split() if word.lower() not in Stop])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df['v2']=df.v2.apply(text)

In [12]:
df.head()


,v2,label_x
0,Go jurong point crazy Available bugis n great ...,0
1,Ok lar Joking wif oni,0
2,Free entry wkly comp win FA Cup final tkts 21s...,1
3,dun say early hor c already say,0
4,Nah think goes usf lives around though,0


In [16]:
df[df.label_x==0].v2.apply(lambda x: [word.lower() for word in x.split()])

0       [go, jurong, point, crazy, available, bugis, n...
1                             [ok, lar, joking, wif, oni]
3                 [dun, say, early, hor, c, already, say]
4          [nah, think, goes, usf, lives, around, though]
6       [even, brother, like, speak, treat, like, aids...
                              ...                        
5565                                           [huh, lei]
5568                   [ì, b, going, esplanade, fr, home]
5569                     [pity, mood, soany, suggestions]
5570    [guy, bitching, acted, like, id, interested, b...
5571                                   [rofl, true, name]
Name: v2, Length: 4825, dtype: object

In [17]:
df[df.label_x==1].v2.apply(lambda x: [word.lower() for word in x.split()])

2       [free, entry, wkly, comp, win, fa, cup, final,...
5       [freemsg, hey, darling, 3, weeks, word, back, ...
8       [winner, valued, network, customer, selected, ...
9       [mobile, 11, months, r, entitled, update, late...
11      [six, chances, win, cash, 100, 20000, pounds, ...
                              ...                        
5537    [want, explicit, sex, 30, secs, ring, 02073162...
5540    [asked, 3mobile, 0870, chatlines, inclu, free,...
5547    [contract, mobile, 11, mnths, latest, motorola...
5566    [reminder, o2, get, 250, pounds, free, call, c...
5567    [2nd, time, tried, contact, å£750, pound, priz...
Name: v2, Length: 747, dtype: object

In [21]:
from sklearn.model_selection import train_test_split
X=df.v2
y=df.label_x

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

In [23]:
X_test.shape

(1393,)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

vect=CountVectorizer()
vect.fit(X_train)

X_train_dtm=vect.transform(X_train)
X_train_dtm=vect.fit_transform(X_train)

X_test_dtm=vect.transform(X_test)

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train_dtm)
tfidf_transformer.transform(X_train_dtm)

<4179x7931 sparse matrix of type '<class 'numpy.float64'>'
	with 34503 stored elements in Compressed Sparse Row format>

In [28]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

nb.fit(X_train_dtm,y_train)

MultinomialNB()

In [30]:
from sklearn import metrics

y_pred_class=nb.predict(X_test_dtm)


print(metrics.accuracy_score(y_test,y_pred_class))

metrics.confusion_matrix(y_test,y_pred_class)

0.9798994974874372


array([[1195,    7],
       [  21,  170]], dtype=int64)

In [32]:
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
metrics.roc_auc_score(y_test,y_pred_prob)

0.9764114782517793